<a href="https://colab.research.google.com/github/joaoBernardinoo/formas-research/blob/main/atividade_01_formas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pickle
import nltk
import pandas as pd
import numpy as np
import spacy
import spacy.cli
from sklearn.model_selection import train_test_split

In [7]:
spacy.cli.download("pt_core_news_sm")
nlp = spacy.load("pt_core_news_sm")

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
!pip install conllu
!wget http://marlovss.work.gd:8080/tomorrow/aula2/bosque.conllu

--2024-10-25 01:05:05--  http://marlovss.work.gd:8080/tomorrow/aula2/bosque.conllu
Resolving marlovss.work.gd (marlovss.work.gd)... 177.180.148.12
Connecting to marlovss.work.gd (marlovss.work.gd)|177.180.148.12|:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11291250 (11M)
Saving to: ‘bosque.conllu.1’

bosque.conllu.1     100%[===================>]  10.77M  3.29MB/s    in 3.3s    

2024-10-25 01:05:08 (3.29 MB/s) - ‘bosque.conllu.1’ saved [11291250/11291250]



In [9]:
import conllu
import itertools as it

class AttributeDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

class CoNLLU:
   def __init__(self, files):
      self.words = []
      self.sentences = []
      for f in files:
         parsed = conllu.parse(open(f).read())
         sents = [[AttributeDict(form = token['form'], lemma=token['lemma'],pos=token['upos'],feats=token['feats']) for token in tokenlist if token['upos']!='_'] for tokenlist in parsed]
         self.sentences.extend(sents)
         self.words.extend([word for sent in sents for word in sent])
      self.pos_tags = set([word.pos for word in self.words])
      self.feats_dict ={pos:set(it.chain.from_iterable([list(word.feats.keys()) for word in self.words if word.pos==pos and word.feats!= None])) for pos in self.pos_tags}


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
bosque = CoNLLU(files=["bosque.conllu"])

In [38]:
train_data, temp_data = train_test_split(bosque.sentences, test_size=0.1, random_state=42) # 90% train, 10% temp
patch_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42) # Split the 10% into 5% patch and 5% test

print(f"Training data size: {len(train_data)}")
print(f"Patch data size: {len(patch_data)}")
print(f"Test data size: {len(test_data)}")

Training data size: 6316
Patch data size: 351
Test data size: 351


In [47]:
from nltk.probability import FreqDist
suffixes = set([word.form.lower()[-3:] for word in bosque.words])


In [48]:
try:
    with open('/content/drive/MyDrive/Colab Notebooks/suf_to_tag.pkl', 'rb') as f:
        suf_to_tag = pickle.load(f)
except FileNotFoundError:
    print("Arquivo nao encontrado, extraindo os sufixos...")
    suf_to_tag = {suf: FreqDist([word.pos for word in bosque.words if word.form.lower()[-3:] == suf]).max() for suf in suffixes}

    with open('/content/drive/MyDrive/Colab Notebooks/suf_to_tag.pkl', 'wb') as f:
        pickle.dump(suf_to_tag, f)

In [40]:
rules = {
    'ADJ': [],
    'ADP': [],
    'ADV': [],
    'AUX': [],
    'CCONJ': [],
    'DET': [],
    'INTJ': [],
    'NOUN': [],
    'NUM': [],
    'PART': [],
    'PRON': [],
    'PROPN': [],
    'PUNCT': [],
    'SCONJ': [],
    'SYM': [],
    'VERB': [],
    'X': []
}

df = pd.DataFrame(list(rules.items()), columns=['pos_tag', 'token'])
df['token'] = df['token'].apply(set)


In [42]:
df.head()

,pos_tag,token
0,ADJ,{}
1,ADP,{}
2,ADV,{}
3,AUX,{}
4,CCONJ,{}


In [43]:
def tag(tokens):
  tagged = []
  for token in tokens:
    if token.lower()[-3:] in suffixes:
       tagged.append((token,suf_to_tag[token.lower()[-3:]]))
    else:
       tagged.append((token,"_"))
  return tagged

In [34]:
def accuracy(predicted,gold):

   acertos = len([predicted[i][j][1] for i in range(len(gold)) for j in range(len(gold[i])) if predicted[i][j][1]==gold[i][j][1]])
   totais = sum([len(sent) for sent in gold])
   return acertos/totais

def abrangencia(predicted,gold):
  tagged_tokens = 0

  for sent in predicted:
    for _, predicted_tag in sent:
      if predicted_tag != "_":
        tagged_tokens += 1
  total_tokens = 0

  for sent in gold:
    for _, gold_tag in sent:
      if gold_tag != "_":
        total_tokens += 1
  return tagged_tokens / total_tokens

def F(predicted,gold):
  return 2 * (abrangencia(predicted,gold) * accuracy(predicted,gold)) / (abrangencia(predicted,gold) + accuracy(predicted,gold))

1167

In [33]:
!wget http://marlovss.work.gd:8080/tomorrow/aula2/test.conllu

--2024-10-25 01:10:15--  http://marlovss.work.gd:8080/tomorrow/aula2/test.conllu
Resolving marlovss.work.gd (marlovss.work.gd)... 177.180.148.12
Connecting to marlovss.work.gd (marlovss.work.gd)|177.180.148.12|:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819980 (1.7M)
Saving to: ‘test.conllu.3’

test.conllu.3       100%[===================>]   1.74M  1.33MB/s    in 1.3s    

2024-10-25 01:10:16 (1.33 MB/s) - ‘test.conllu.3’ saved [1819980/1819980]



In [31]:
test = CoNLLU(files=["test.conllu"])
test_sents = [[word.form for word in sent] for sent in test.sentences]
gold = [[(word.form.lower(),word.pos) for word in sent] for sent in test.sentences]
predicted = [tag(sent) for sent in test_sents]

In [ ]:
def validate():
  gold = [[(word.form.lower(),word.pos) for word in sent] for sent in test.sentences]
  predicted = [tag(sent) for sent in test_sents]
  return {
        'accuracy': accuracy(predicted,gold),
        'coverage': abrangencia(predicted,gold),
        "F" : F(predicted,gold)
}

In [ ]:
def humanTaggingHelper():
    tagged_sent = next((sent for sent in predicted if any(tag == "_" for _, tag in sent)), None)
    if tagged_sent:
        print(tagged_sent)
        tokens = [token for token, _ in tagged_sent]
        doc = spacy.tokens.Doc(nlp.vocab, words=tokens)
        for token, (text, tag) in zip(doc, tagged_sent):
            if tag != "_":
                token.pos_ = tag
            else:
                token.tag_ = tag
        displacy.render(doc, style='dep', jupyter=True, options={'distance': 70})

In [ ]:
def add_rule_to_df(df, category, token):
    token_lower = token.lower()
    idx = df[df['pos_tag'] == category].index[0]
    df.iloc[idx,1].add(token_lower)
    print(f"'{token_lower}' adicionado à regra {category}")

In [ ]:
from spacy import displacy
categories = df.iloc[:,0]

def humanTaggingHelper():
    tagged_sent = next((sent for sent in predicted if any(tag == "_" for _, tag in sent)), None)
    if tagged_sent:
        print(tagged_sent)
        tokens = [token for token, _ in tagged_sent]
        doc = spacy.tokens.Doc(nlp.vocab, words=tokens)
        for token, (text, tag) in zip(doc, tagged_sent):
            if tag != "_":
                token.pos_ = tag
            else:
                token.tag_ = tag
        displacy.render(doc, style='dep', jupyter=True, options={'distance': 70})

        print("Mapeamento de Categorias:")
        for i, category in enumerate(categories):
            print(f"{i} - {category}")

        for token, (text, tag) in zip(doc, tagged_sent):
            if tag == "_":
                new_tag = input(f"Qual é a categoria de '{text}'? (Q para sair) ").strip().upper()
                if new_tag == "Q":
                    print("Encerrando o programa.")
                    return
                if new_tag.isdigit() and int(new_tag) < len(categories):
                    category = categories[int(new_tag)]
                    add_rule_to_df(df, category, text)
                else:
                    print(f"Entrada inválida ou categoria não encontrada.")

In [ ]:
predicted[0]

In [ ]:
humanTaggingHelper()

In [ ]:
df

In [ ]:
for sent in predicted:
    if any(predicted_tag == "_" for _,predicted_tag in sent):
      print(str(sent))
      break

In [ ]:
test_sents[0]
sent = ""
for word in test_sents[0]:
  sent += word + " "
print(sent)

## Fazendo nosso lematizador e analisador de flexões

O lematizador e a análise flexional podem ser realizados em conuunto, uma vez que determinar as flexões nos permitem "desfazê-las", i.e. obter uma versão "normalizada" do item lexical